In [2]:
from src.data.utils                 import setup_dataloader
from src.models.transformer_wrapper import TransformerWrapper

In [ ]:
# DATA
TRAIN_PATH = "data/TRAIN_RELEASE_3SEP2025/train_subtask1.csv"

# TRANSFOMER
TOKENIZER_PATH = "bert-base-uncased"
MODEL_PATH = "bert-base-uncased"

df = pd.read_csv(TRAIN_PATH)

transfomer = TransformerWrapper(
    tokenizer_path=TOKENIZER_PATH,
    model_path=MODEL_PATH,
)

dataloader, dataset = setup_dataloader(
    csv_path=TRAIN_PATH,
    tokenizer_path=TOKENIZER_PATH,
    max_text_length=512,
    batch_size=16,
    shuffle=True
)